Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"
%matplotlib inline

/Users/gianlucafugante/anaconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Available frameworks:
Qiskit
D-Wave Ocean


# The Ising model

**Exercise 1** (1 point). The Ising model is a basic model of statistical mechanics that explains a lot about how quantum optimizers work. Its energy is described by its Hamiltonian:

$$ H=-\sum_{<i,j>} J_{ij} \sigma_i \sigma_{j} - \sum_i h_i \sigma_i$$.

Write a function that calculates this energy amount for a linear chain of spins. The function takes three arguments: `J`, `h`, and `σ`, corresponding to the coupling strengths, the onsite field at each site, and the specific spin configuration

In [7]:
def calculate_energy(J, h, σ):
    ###
    ### YOUR CODE HERE
    ###

    magnetic_interaction = np.array(h).dot(np.array(σ))
    
    coupling_interaction = 0.
    
    try:
        for i in range(len(J)):
            coupling_interaction = coupling_interaction + ( J[i] * σ[i] * σ[i+1] )
        
        return np.negative(coupling_interaction + magnetic_interaction)
    
    except:
        print("An exception occurred")
        return False
    

In [8]:
J = [1.0, -1.0]
σ = [+1, -1, +1]
h = [0.5, 0.5, 0.4]
assert abs(calculate_energy(J, h, σ)+0.4) < 0.01
J = [-1.0, 0.5, 0.9]
σ = [+1, -1, -1, -1]
h = [4, 0.2, 0.4, 0.7]
assert abs(calculate_energy(J, h, σ)+5.1) < 0.01

**Exercise 2** (2 points). The sign of the coupling defines the nature of the interaction, ferromagnetic or antiferromagnetic, corresponding to positive and negative $J$ values, respectively. Setting the couplings to zero, we have a non-interacting model. Create an arbitrary antiferromagnetic model on three sites with no external field. Define the model through variables `J` and `h`. Iterate over all solutions and write the optimal one in a variable called `σ`. If the optimum is degenerate, that is, you have more than one optimal configuration, keep one.

In [49]:
import itertools
###
### YOUR CODE HERE
###

h = [0., 0., 0.]
J = [-1., -1.]
all_σ_permutations = [p for p in itertools.product([1.,-1.], repeat=3)]


solutions = []

for σ_i in all_σ_permutations:
    energy = calculate_energy(J, h, σ_i)
    solutions.append((energy, σ_i))
    

solutions.sort(key=lambda x:x[0])
σ = solutions[0][1]
print(solutions[0][0], σ)

-2.0 (1.0, -1.0, 1.0)


In [50]:
assert all([J_i < 0 for J_i in J])
assert all([h_i == 0 for h_i in h])
assert len(J) == 2
assert len(h) == 3
assert all([σ[i]*σ[i+1] == -1 for i, _ in enumerate(J)]), "The configuration is not the optimum of an antiferromagnetic system"

**Exercise 3** (1 point). Iterating over all solutions is clearly not efficient, since there are exponentially many configurations in the number of sites. From the perspective of computer science, this is a combinatorial optimization problem, and it is a known NP-hard problem. Many heuristic methods have been invented to tackle the problem. One of them is simulated annealing. It is implemented in dimod. Create the same antiferromagnetic model in dimod as above. Keep in mind that dimod uses a plus and not a minus sign in the Hamiltonian, so the sign of your couplings should be reversed. Store the model in an object called `model`, which should be a `BinaryQuadraticModel`.

In [53]:
###
### YOUR CODE HERE
###

h = {0: 0, 1: 0, 2: 0}
J = {(0, 1): 1., (1, 2): 1.}

model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

The simulated annealing solver requires us to define the couplings as a dictionary between spins, and we must also pass the external field values as a dictionary. The latter is all zeros for us.

In [54]:
assert isinstance(model, dimod.binary_quadratic_model.BinaryQuadraticModel), "Wrong model type"
assert model.vartype == dimod.SPIN, "Wrong variables: binary model instead of spin system"
assert all([J_i > 0 for J_i in J.values()]), "The model is not antiferromagnetic"

**Exercise 4** (1 point). Sample the solution space a hundred times and write the response in an object called `response`.

In [56]:
###
### YOUR CODE HERE
###
sampler = dimod.RandomSampler()
response = sampler.sample(model, num_reads=100)
print(response)

    0  1  2 energy num_oc.
0  +1 -1 +1   -2.0       1
2  +1 -1 +1   -2.0       1
9  +1 -1 +1   -2.0       1
14 -1 +1 -1   -2.0       1
20 +1 -1 +1   -2.0       1
21 +1 -1 +1   -2.0       1
26 -1 +1 -1   -2.0       1
32 +1 -1 +1   -2.0       1
35 +1 -1 +1   -2.0       1
36 -1 +1 -1   -2.0       1
39 -1 +1 -1   -2.0       1
42 +1 -1 +1   -2.0       1
46 -1 +1 -1   -2.0       1
47 -1 +1 -1   -2.0       1
54 +1 -1 +1   -2.0       1
58 -1 +1 -1   -2.0       1
62 -1 +1 -1   -2.0       1
66 -1 +1 -1   -2.0       1
70 -1 +1 -1   -2.0       1
72 +1 -1 +1   -2.0       1
73 -1 +1 -1   -2.0       1
78 -1 +1 -1   -2.0       1
85 +1 -1 +1   -2.0       1
88 +1 -1 +1   -2.0       1
94 -1 +1 -1   -2.0       1
4  -1 +1 +1    0.0       1
5  -1 +1 +1    0.0       1
6  +1 -1 -1    0.0       1
8  -1 -1 +1    0.0       1
10 -1 +1 +1    0.0       1
11 -1 +1 +1    0.0       1
12 -1 -1 +1    0.0       1
13 +1 +1 -1    0.0       1
17 -1 -1 +1    0.0       1
24 +1 -1 -1    0.0       1
25 -1 -1 +1    0.0       1
2

In [57]:
assert len(response) == 100, "Not the correct number of samples"
sample = response.first.sample
assert all([sample[i]*sample[i+1] == -1 for i, _ in enumerate(J.values())]), "The optimal configuration is not antiferromagnetic"

# The transverse-field Ising model

**Exercise 5** (1 point). Adiabatic quantum computation and quantum annealing rely on quantum variants of the classical Ising model, and so do some variational algorithms like the quantum approximate optimization algorithm. To understand the logic behind these simple quantum-many body systems, first let us take another look at the classical Ising model, but write the Hamiltonian of the system in the quantum mechanical formalism, that is, with operators:

$$ H=-\sum_{<i,j>} J_{ij} \sigma^Z_i \sigma^Z_{j} - \sum_i h_i \sigma^Z_i$$.

Assume that you only have two sites. Create the Hamiltonian $H=-\sigma^Z_1\sigma^Z_2$ as a $4\times 4$ numpy array called `H`. Recall that on a single site, $\sigma^Z$ is the Pauli-Z matrix $\begin{bmatrix}1 & 0\\ 0& -1\end{bmatrix}$.

In [138]:
###
### YOUR CODE HERE
###
sz_1 = np.matrix([[1,0],[0,-1]])
sz_2 = np.matrix([[1,0],[0,-1]])

H = np.kron(sz_1, sz_2).dot(-1)
print(H)

[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]


In [139]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Now take a look at the eigenvector corresponding to the two smallest eigenvalues (both are -1):

In [142]:
eigenvalues, eigenvectors = np.linalg.eigh(H)
print(eigenvectors[:, 0:1])
print(eigenvectors[:, 1:2])

print(np.linalg.eigh(H))

[[1.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [1.]]
(array([-1., -1.,  1.,  1.]), matrix([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]]))


This is just the $|00\rangle$ and $|11\rangle$ states, confirming our classical intuition that in this ferromagnetic case (J=1), the two spins should be aligned to get the minimum energy, the ground state energy.

We copy the function that calculates the energy expectation value $<H>$ of a Hamiltonian $H$ and check the expectation value in the $|00\rangle$ state:

In [143]:
def calculate_energy_expectation(state, hamiltonian):
    return float(np.dot(state.T.conj(), np.dot(hamiltonian, state)).real)

ψ = np.kron([[1], [0]], [[1], [0]])
calculate_energy_expectation(ψ, H)

-1.0

It comes to -1.

**Exercise 6** (1 point). If we add a term that does not commute with the Pauli-Z operator, the Hamiltonian will display non-classical effects. Add a Pauli-X term to both sites, so your total Hamiltonian will be $H=-\sigma^Z_1\sigma^Z_2-\sigma^X_1-\sigma^X_2$, in the object `H`.

In [144]:
###
### YOUR CODE HERE
###

sx_1 = np.matrix([[0,1],[1,0]])
sx_2 = np.matrix([[0,1],[1,0]])



H_x1 = np.kron(sx_1, np.identity(2)).dot(-1)
H_x2 = np.kron(np.identity(2), sx_2).dot(-1)

H = H + H_x1 + H_x2

print(H)

[[-1. -1. -1.  0.]
 [-1.  1.  0. -1.]
 [-1.  0.  1. -1.]
 [ 0. -1. -1. -1.]]


In [145]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


If you take a look at the matrix of the Hamiltonian, it has off-diagonal terms:

In [146]:
H

matrix([[-1., -1., -1.,  0.],
        [-1.,  1.,  0., -1.],
        [-1.,  0.,  1., -1.],
        [ 0., -1., -1., -1.]])

The energy expectation value in the $|00\rangle$ is not affected, the transverse field only lowers the ground state energy:

In [148]:
ψ = np.kron([[1], [0]], [[1], [0]])
print(ψ)
calculate_energy_expectation(ψ, H)

[[1]
 [0]
 [0]
 [0]]


-1.0

**Exercise 7** (1 point). Is this the ground state energy? Use the eigenvector corresponding to the smallest eigenvalue and calculate the expectation value of it. Store the value in a variable called `energy_expectation_value`.

In [154]:
###
### YOUR CODE HERE
###
energy_expectation_value = calculate_energy_expectation(eigenvectors[:, 0:1], H)
print(energy_expectation_value)
energy_expectation_value = calculate_energy_expectation(eigenvectors[:, 1:2], H)
print(energy_expectation_value)

-1.0
-1.0


In [155]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Naturally, this value also corresponds to the lowest eigenvalue and indeed, this is the ground state energy. So by calculating the eigendecomposition of the typically non-diagonal Hamiltonian, we can extract both the ground state and its energy. The difficulty comes from the exponential scaling of the matrix representing the Hamiltonian as a function of the number of sites. This is the original reason going back to the early 1980s to build a quantum computer: this device would implement (or simulate) the Hamiltonian in hardware. Say, a couple of hundred spins would be beyond the computational capacity of supercomputers, but having the physical spins and being able to set a specific Hamiltonian, we can extract quantities of interest, such the ground state.